# ALERT SYSTEM

ADX: https://www.investopedia.com/articles/trading/07/adx-trend-indicator.asp

Cron Job: https://crontab.guru/
*/2 * * * *

In [2]:
import pandas as pd
import pandas_ta as ta
import ccxt, yfinance
import requests

In [3]:
exchange = ccxt.binance()

In [4]:
symbols = pd.read_csv("symbols.csv")
symbols = pd.DataFrame(symbols, columns = ["symbol"])
symbols = symbols["symbol"].values.tolist()

In [30]:
for symbol in symbols:
    bars = exchange.fetch_ohlcv(symbol, timeframe = "5m", limit = 500)
    df = pd.DataFrame(bars, columns = ["time", "open", "high", "low", "close", "volume"])
    df['time'] = pd.to_datetime(df['time'], unit = 'ms')    
    
    # technical indicators
    adx = df.ta.adx()
    macd = df.ta.macd(fast = 14, slow = 28)
    rsi = df.ta.rsi()

    # create composite df
    df = pd.concat([df, adx, macd, rsi], axis = 1)

    # grab the last value
    last_row = df.iloc[-1]
    cap_time = last_row[0]

    # hook to discord channel
    WEBHOOK_URL = "https://discord.com/api/webhooks/982327105041875004/w_ume6BIxlHe3C9jgRUrUXWSFn3oiLcvpb2118vsNh8iZdRGHxD5LqYDer2cGwsjd_sL"

    # logic
    if last_row["ADX_14"] > 25:
        if last_row["DMP_14"] > last_row["DMN_14"]:
            message = f"{cap_time} STRONG UP TREND: {symbol}: The ADX is {last_row['ADX_14']: .2f} +DI {last_row['DMP_14']: .2f} -DI {last_row['DMN_14']: .2f}"
        if last_row["DMN_14"] > last_row["DMP_14"]:
            message = f"{cap_time} STRONG DOWN TREND:{symbol}: The ADX is {last_row['ADX_14']: .2f} +DI {last_row['DMP_14']: .2f} -DI {last_row['DMN_14']: .2f}"

        payload = {
            "username" : "alertbot",
            "content" : message
        }

        print(message)
        #requests.post(WEBHOOK_URL, json = payload)

    if last_row["ADX_14"] < 25:
        message = f"{cap_time} no trend:{symbol}: The ADX is {last_row['ADX_14']: .2f} +DI {last_row['DMP_14']: .2f} -DI {last_row['DMN_14']: .2f}"

        payload = {
            "username" : "alertbot",
            "content" : message
        }
        
        print(message)
        #requests.post(WEBHOOK_URL, json = payload)

time             2022-06-03 17:55:00
open                         1752.58
high                         1753.73
low                          1750.58
close                        1753.15
volume                     1381.9833
ADX_14                      11.49137
DMP_14                     17.093766
DMN_14                     23.802613
MACD_14_28_9               -2.152543
MACDh_14_28_9              -0.319919
MACDs_14_28_9              -1.832625
RSI_14                     45.424982
Name: 499, dtype: object
2022-06-03 17:55:00 no trend:ETH/USDT: The ADX is  11.49 +DI  17.09 -DI  23.80
time             2022-06-03 17:55:00
open                        29509.19
high                        29519.59
low                         29485.89
close                       29519.59
volume                      59.62987
ADX_14                     11.524587
DMP_14                     19.419563
DMN_14                     20.052138
MACD_14_28_9               -2.437977
MACDh_14_28_9                0.88651
MACDs_14